In [52]:
from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, List

In [53]:
# Read all the log files and put their contents into the log_lines dict
import io


LOG_FOLDER = Path("/var/avl_logs/current/log")

# Grab all files in log folder that end in .log
log_files = [f for f in LOG_FOLDER.iterdir() if f.is_file() and f.suffix == '.log']

def read_log(file: Path | str) -> Dict[str, pd.DataFrame]:
    log_strs = {}
    log_dfs = {}
    with open(file, "r") as f:
        data_lines = [line for line in f.readlines() if "[DAT]" in line]
        
        # must have at least names row and units row
        assert len(data_lines) >= 2 

        for line in data_lines:
            stream = line.strip().split(' ')[3]
            if stream not in log_strs:
                log_strs[stream] = []
            data = line.strip().split(' ')
            time = data[0].replace("[", "").replace("]", "")
            log_strs[stream].append([time] + data[4:])

        for stream, data in log_strs.items():
            names = data.pop(0)[1:]
            units = data.pop(0)[1:]
            col_names = ["time"] + [f"{name}_{unit}" for name, unit in zip(names, units)]
            log_dfs[stream] = pd.DataFrame(data, columns=col_names)
            print(stream)
            display(log_dfs[stream])

    return log_dfs

# Read each log file into a DataFrame
# log_data: Dict[str, pd.DataFrame] = {}
# for log_file in log_files:
#     ...
    

read_log('/var/avl_logs/current/log/auv_dynamics_node.log')

[nu]


,time,u_m/s,v_m/s,w_m/s,p_rad/s,q_rad/s,r_rad/s
0,1679338479.348389626,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1679338479.350642681,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1679338479.353406191,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1679338479.354749441,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1679338479.356523514,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
5068,1679338492.256108046,1.950200,0.004147,0.009189,0.001480,0.000763,-0.004456
5069,1679338492.258646727,1.948167,0.004141,0.009199,0.001476,0.000787,-0.004446
5070,1679338492.260325670,1.946138,0.004135,0.009209,0.001472,0.000811,-0.004435
5071,1679338492.261921883,1.944114,0.004129,0.009220,0.001468,0.000834,-0.004425


[eta]


,time,x_m,y_m,z_m,\phi_rad,\theta_rad,\psi_rad
0,1679338479.348571539,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
1,1679338479.350677490,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
2,1679338479.353446245,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
3,1679338479.354799271,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
4,1679338479.356557369,0.000000,0.000000,0.000000,0.000000,0.000000,0.785398
...,...,...,...,...,...,...,...
5068,1679338492.256139278,55.524419,20.288123,-14.901504,-0.001384,0.274225,0.233771
5069,1679338492.258682251,55.542677,20.292513,-14.906691,-0.001381,0.274233,0.233725
5070,1679338492.260380268,55.560917,20.296898,-14.911872,-0.001379,0.274241,0.233679
5071,1679338492.261955500,55.579137,20.301278,-14.917049,-0.001377,0.274249,0.233633


[tau]


,time,thrust_N,\tau_1_?,\tau_2_?,\tau_3_?,\tau_4_?,\tau_5_?
0,1679338479.348640203,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000
1,1679338479.350700378,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000
2,1679338479.353471756,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000
3,1679338479.354824543,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000
4,1679338479.356580496,0.000000,-0.000000,-0.000000,0.000000,-0.000000,0.000000
...,...,...,...,...,...,...,...
5068,1679338492.256158352,0.000000,0.079223,0.741327,0.000000,0.761569,-0.081387
5069,1679338492.258698463,0.000000,0.079058,0.739782,0.000000,0.759982,-0.081217
5070,1679338492.260405779,0.000000,0.078894,0.738242,0.000000,0.758399,-0.081048
5071,1679338492.261980057,0.000000,0.078730,0.736707,0.000000,0.756822,-0.080879


[nu_dot]


,time,u\_dot_m/s^2,v\_dot_m/s^2,w\_dot_m/s^2,p\_dot_rad/s^2,q\_dot_rad/s^2,r\_dot_rad/s^2
0,1679338479.348682165,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1679338479.350725174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1679338479.353497744,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1679338479.354841709,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1679338479.356604338,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
5068,1679338492.256180286,-0.203306,-0.000614,0.001022,-0.000416,0.002481,0.001051
5069,1679338492.258714914,-0.202881,-0.000614,0.001021,-0.000405,0.002385,0.001041
5070,1679338492.260431528,-0.202459,-0.000614,0.001020,-0.000394,0.002291,0.001031
5071,1679338492.262009144,-0.202037,-0.000614,0.001019,-0.000383,0.002198,0.001021
